In [1]:
import numpy as np
import pandas as pd
from datetime import date, datetime
import sqlite3

In [2]:
def gen_claim(count):
    columnNames = ['claimId', 'ccn', 'providerType', 'lineNumber', 'allowedAmount', 'procedureCode', 'diagnosis', 'fromDate']
    providerTypes = ['inpatient', 'snf', 'home health', 'hha', 'null', 'outpatient']
    procedureCodes = ['CAT THERAPY', 'INSULIN', 'PHYSICAL THERAPY', 'LIGHT THERAPY', 'INSULINE']
    diagnoses = ['DIABETES', 'LONELINESS', 'SADNESS', 'LACK OF EXERCISE', 'DBTS']
    prevClaimID = 0
    data = []
    while (prevClaimID < count):
        #providerType = random from providerType, null = outpatient, hha = home health 
        newProviderType = np.random.default_rng().choice(providerTypes)
        #lineNumber = 1 unless home health claim (home health or hha) which can have up to 10. 
        #    Has the same claimID, providerType, and ccn.
        if newProviderType in ['home health','hha']:
            newLineNumber = np.random.default_rng().integers(2,11)
        else:
            newLineNumber = 1
        #ccn = random 6 digits, starts with 4 iff outpatient provider type
        if newProviderType in ['null','outpatient']:
            newCcn = np.random.default_rng().integers(400000,500000)
        else:
            newCcn = np.random.default_rng().choice([np.random.default_rng().integers(400000), np.random.default_rng().integers(500000,1000000)])
        for i in range(0, newLineNumber):
        #   claimId = 'claim_id_#', where there are 5000 claims, unique unless multi line claim
            if i > 0:
                newClaimID = prevClaimID
            else:
                newClaimID = prevClaimID + 1
            prevClaimID = newClaimID
        #
        #   procedureCode = random procedure
            newProcedureCode = np.random.default_rng().choice(procedureCodes)
        #   diagnosis = random diagnosis
            newDiagnosis = np.random.default_rng().choice(diagnoses)
        #   fromDate = random date between 2010 and 2020. If lineNumber > 1, subsequent entries shoudl be one day more
            if i > 0:
                newDate = prevDate + 1 
                #Not allowing for dates to be outside the given range
                if np.datetime64('2021-01-01') <= newDate:
                    break
            else:
                newDate = np.datetime64('2010') + np.timedelta64(np.random.default_rng().integers(4018), 'D')
            prevDate = newDate

        #   allowedAmount = snf, inpatient, outpatient = lognormal; home health = uniform. 
        #   means: snf > inpatient > outpatient >> home health
            if newProviderType in 'snf':
                mean = 9
            if newProviderType in 'inpatient':
                mean = 8
            if newProviderType in ['outpatient','null']:
                mean = 7
            if newProviderType in ['home health','hha']:
                mean = 100
        #   if insulin, mean * 10%
            if newProcedureCode in ['INSULIN','INSULINE']:
                mean *= .1
        #   use `numpy.random.default_rng().lognormal` or `numpy.random.default_rng().uniform`.
            if newProviderType in ['inpatient', 'snf', 'null', 'outpatient']:
                newAllowedAmount = np.random.default_rng().lognormal(mean, .2)
            else: 
                newAllowedAmount = np.random.default_rng().uniform(mean,.1)
            
            #Append each line of the claim as generated
            data.append([f'claim_id_{newClaimID}', f'{newCcn:06d}', f'{newProviderType}', f'line_number_{i+1}', f'{newAllowedAmount}', f'{newProcedureCode}', f'{newDiagnosis}', f'{newDate}'])
    
    return pd.DataFrame(data, columns = columnNames)

In [3]:
def transform_data(data):
    df = data.copy()
    #update columns to snake_case
    df.columns = [''.join(['_'+c.lower() if c.isupper() else c for c in x]).lstrip('_') for x in df.columns]
    #claimID does not need adjusted
    #ccn does not need adjusted
    #provider type needs types updated: inpatient -> ip, home health -> hha, outpatient & null -> op
    df.loc[df['provider_type'].isin(['outpatient','null']), ['provider_type']] = 'op'
    df.loc[df['provider_type'].isin(['home health']), ['provider_type']] = 'hha'
    df.loc[df['provider_type'].isin(['inpatient','null']), ['provider_type']] = 'ip'
    #line number needs digits only
    df['line_number'] = df['line_number'].str.lstrip('line_number_')
    df['line_number'] = df['line_number'].astype(int)
    #allowed amount convert to float
    df["allowed_amount"] = df["allowed_amount"].astype(float)
    #procedure code convert INSULINE -> INSULIN
    df.loc[df["procedure_code"] == 'INSULINE', ["procedure_code"]]='INSULIN'
    #diagnosis convert DBTS -> DIABETES
    df.loc[df["diagnosis"] == 'DBTS', ["diagnosis"]]='DIABETES'
    #date needs converted to date format
    df["from_date"] = [date.fromisoformat(x) for x in df["from_date"]]
    return df

In [4]:
def flag_data(data):
    df = transform_data(data)
    #create boolean column "is_diabetic"
    df['is_diabetic'] = [True if x == 'DIABETES' else False for x in df['diagnosis']]
    #create boolean column 'is_lonely'
    df['is_lonely'] = [True if x == 'LONELINESS' else False for x in df['diagnosis']]
    #creat boolean column 'given_insulin'
    df['given_insulin'] = [True if x == 'INSULIN' else False for x in df['procedure_code']]
    #create boolean column 'high cost', flag if allowed_amount >= 90th percentile within provider and procedure code
    high = df.groupby(['procedure_code', 'provider_type'])['allowed_amount'].transform(lambda x: np.percentile(x,90))
    df['high_cost'] = np.where(df['allowed_amount'] >= high, True, False)
    #boolean column 'outlier_cost_inpatient', flag if allowed_amount >= 99th percentile cost withing inpatient
    #error if no inpatient provider types exist
    try:
        outlier = np.percentile(df['allowed_amount'][df['provider_type'] == 'ip'], 99)
        df['outlier_cost_inpatient'] = np.where((df['allowed_amount'] > outlier) & (df['provider_type'] == 'ip'), True, False)
    except:
        df['outlier_cost_inpatient'] = False
    #created_at datetime, between 2010-01-01 - 2020-12-31
    input_str = f"{np.datetime64('2010') + np.timedelta64(np.random.default_rng().integers(4018), 'D')}"
    dt_object = datetime.strptime(input_str, '%Y-%m-%d')
    df['created_at'] = dt_object
    #active True for all rows when generated
    df['active'] = True
    #valid_from & valid_thru 2021-01-01 - 2021-12-31
    df['valid_from'] = date.fromisoformat('2021-01-01')
    df['valid_thru'] = date.fromisoformat('2021-12-31')
    
    
    return df

In [5]:
schema1 = gen_claim(5000)

In [6]:
schema2 = flag_data(schema1)

In [7]:
con = sqlite3.connect('signify_data_engineering.db')
cur = con.cursor()

In [8]:
#Check if table already exists, and create it if it does not
listOfTables = cur.execute(
  """select name from sqlite_master where type='table'
  AND name='raw_claims_history'; """).fetchall()
 
if listOfTables == []:
    raw_claims_history = ''' create table raw_claims_history(
        claimId text not null,
        ccn text not null,
        providerType text,
        lineNumber text not null,
        allowedAmount text not null,
        procedureCode text not null,
        diagnosis text not null,
        fromDate text not null
        );   
        '''
    cur.execute(raw_claims_history)

In [9]:
#Pandas function to export dataframe to SQL table automatically
#schema1.to_sql("raw_claims_history", con, if_exists="append", index = False)
for ind, row in schema1.iterrows():
    cur.execute('''insert into raw_claims_history (
    claimId,
    ccn,
    providerType,
    lineNumber,
    allowedAmount,
    procedureCode,
    diagnosis,
    fromDate
    ) values (?,?,?,?,?,?,?,?)''',row)

In [10]:
#Check if table already exists, and create it if it does not
listOfTables = cur.execute(
  """select name from sqlite_master where type='table'
  AND name='claims_history'; """).fetchall()
 
if listOfTables == []:
    claims_history = ''' create table claims_history(
        claim_id text not null,
        ccn text not null,
        provider_type text not null,
        line_number integer not null,
        allowed_amount float not null,
        procedure_code text not null,
        diagnosis text not null,
        from_date date not null,
        is_diabetic bool not null,
        is_lonely bool not null,
        given_insulin bool not null,
        high_cost bool not null,
        outlier_cost_inpatient bool,
        created_at datetime not null,
        active bool not null,
        valid_from date not null,
        valid_thru date not null
        );
        '''
    cur.execute(claims_history)

In [11]:
cur.execute('update claims_history set active = 0')
#Pandas function to export dataframe to SQL table automatically
#schema2.to_sql("claims_history", con, if_exists="replace", index = False)
#As sqlite doesn't accept the datetime64 object the datetime is stored as, its temporarily changed to a string
schema2['created_at'] = np.vectorize(str)(schema2['created_at'])
for ind, row in schema2.iterrows():
    cur.execute('''insert into claims_history (
    claim_id,
    ccn,
    provider_type,
    line_number,
    allowed_amount,
    procedure_code,
    diagnosis,
    from_date,
    is_diabetic,
    is_lonely,
    given_insulin,
    high_cost,
    outlier_cost_inpatient,
    created_at,
    active,
    valid_from,
    valid_thru
    ) values (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)''',row)
schema2['created_at'] = np.vectorize(datetime.strptime)(schema2['created_at'], '%Y-%m-%d %H:%M:%S')

In [12]:
con.commit()

In [13]:
cur.close()
con.close()